# Detección de transacciones fraudulentas

## Uso de librerías

In [9]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, RobustScaler
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier


## Entrada y exploración de datos

In [ ]:
df = pd.read_csv('dataset/creditcard.csv', dtype={'column_name': 'string'})
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'creditcard.csv'

In [ ]:
print("Información del dataset:")
print(f"Dimensiones: {df.shape}")
print(f"Distribución de clases: {df['Class'].value_counts()}")
print(f"Porcentaje de fraude: {df['Class'].mean()*100:.4f}%")
print(f"\nValores nulos: {df.isnull().sum().sum()}")

Información del dataset:
Dimensiones: (284807, 31)
Distribución de clases: Class
0    284315
1       492
Name: count, dtype: int64
Porcentaje de fraude: 0.1727%

Valores nulos: 0


## Preprocesamiento de datos

### Escalar datos

In [ ]:
columnas_pca = [f'V{i}' for i in range(1, 29)]

print("VERIFICACIÓN DE COMPONENTES PCA:")
print("=" * 50)

for col in columnas_pca:
    media = df[col].mean()
    std = df[col].std()
    print(f"{col}: Media = {media:8.4f}, Std = {std:8.4f}")

VERIFICACIÓN DE COMPONENTES PCA:
V1: Media =   0.0000, Std =   1.9587
V2: Media =   0.0000, Std =   1.6513
V3: Media =  -0.0000, Std =   1.5163
V4: Media =   0.0000, Std =   1.4159
V5: Media =   0.0000, Std =   1.3802
V6: Media =   0.0000, Std =   1.3323
V7: Media =  -0.0000, Std =   1.2371
V8: Media =   0.0000, Std =   1.1944
V9: Media =  -0.0000, Std =   1.0986
V10: Media =   0.0000, Std =   1.0888
V11: Media =   0.0000, Std =   1.0207
V12: Media =  -0.0000, Std =   0.9992
V13: Media =   0.0000, Std =   0.9953
V14: Media =   0.0000, Std =   0.9586
V15: Media =   0.0000, Std =   0.9153
V16: Media =   0.0000, Std =   0.8763
V17: Media =  -0.0000, Std =   0.8493
V18: Media =   0.0000, Std =   0.8382
V19: Media =   0.0000, Std =   0.8140
V20: Media =   0.0000, Std =   0.7709
V21: Media =   0.0000, Std =   0.7345
V22: Media =  -0.0000, Std =   0.7257
V23: Media =   0.0000, Std =   0.6245
V24: Media =   0.0000, Std =   0.6056
V25: Media =   0.0000, Std =   0.5213
V26: Media =   0.0000, Std

In [ ]:
# Verificación general
print(f"\nESTADÍSTICAS GENERALES PCA:")
print(f"Rango de medias: [{df[columnas_pca].mean().min():.4f}, {df[columnas_pca].mean().max():.4f}]")
print(f"Rango de std: [{df[columnas_pca].std().min():.4f}, {df[columnas_pca].std().max():.4f}]")


ESTADÍSTICAS GENERALES PCA:
Rango de medias: [-0.0000, 0.0000]
Rango de std: [0.3301, 1.9587]


Dado que las medias son igual a 0, en el caso de las variables PCA, pero sus desviaciones estándar no son iguales, se necesita escalar esos datos sin centrar.

In [ ]:
escalado_sin_centrar = StandardScaler(with_mean=False)

preprocesador_optimo = ColumnTransformer([
    # PCA: Solo escalar, no centrar (ya están centradas)
    ('pca_features', escalado_sin_centrar, [f'V{i}' for i in range(1, 29)]),
    
    # Time y Amount: RobustScaler completo
    ('robust_features', RobustScaler(), ['Time', 'Amount'])
])

print("ESTRATEGIA APLICADA:")
print("   - V1-V28: StandardScaler(with_mean=False) → Solo escalar")
print("   - Time: RobustScaler → Manejar outliers temporales")  
print("   - Amount: RobustScaler → Manejar outliers monetarios")

🎯 ESTRATEGIA APLICADA:
   - V1-V28: StandardScaler(with_mean=False) → Solo escalar
   - Time: RobustScaler → Manejar outliers temporales
   - Amount: RobustScaler → Manejar outliers monetarios


In [ ]:
# Aplicar el preprocesamiento
X_escalado = preprocesador_optimo.fit_transform(df.drop('Class', axis=1))

# Convertir a DataFrame para verificación
columnas_escaladas = [f'V{i}' for i in range(1, 29)] + ['Time', 'Amount']
df_verificacion = pd.DataFrame(X_escalado, columns=columnas_escaladas)

print("📊 VERIFICACIÓN POST-ESCALADO:")
print("Componentes PCA después del escalado:")
print(df_verificacion[[f'V{i}' for i in range(1, 29)]].std())

📊 VERIFICACIÓN POST-ESCALADO:
Componentes PCA después del escalado:
V1     1.000002
V2     1.000002
V3     1.000002
V4     1.000002
V5     1.000002
V6     1.000002
V7     1.000002
V8     1.000002
V9     1.000002
V10    1.000002
V11    1.000002
V12    1.000002
V13    1.000002
V14    1.000002
V15    1.000002
V16    1.000002
V17    1.000002
V18    1.000002
V19    1.000002
V20    1.000002
V21    1.000002
V22    1.000002
V23    1.000002
V24    1.000002
V25    1.000002
V26    1.000002
V27    1.000002
V28    1.000002
dtype: float64


### Dividir el conjunto de datos

In [ ]:
X = df.drop('Class', axis=1)
y = df['Class']  

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    stratify=y,
    random_state=42
)

print("DIVISIÓN ESTRATIFICADA 70-30:")
print(f"Dataset original - Clase 1: {y.mean():.4f}%")
print(f"Train - Clase 1: {y_train.mean():.4f}%")
print(f"Test - Clase 1: {y_test.mean():.4f}%")

✅ DIVISIÓN ESTRATIFICADA 70-30:
Dataset original - Clase 1: 0.0017%
Train - Clase 1: 0.0017%
Test - Clase 1: 0.0017%


## Selección de características

In [ ]:
X_train_preprocesado = preprocesador_optimo.fit_transform(X_train)
X_test_preprocesado = preprocesador_optimo.transform(X_test)

# Nombres de las características después del preprocesamiento
nombres_caracteristicas = [f'V{i}' for i in range(1, 29)] + ['Time', 'Amount']

print(f"📊 Shape de X_train preprocesado: {X_train_preprocesado.shape}")
print(f"📊 Shape de X_test preprocesado: {X_test_preprocesado.shape}")


📊 Shape de X_train preprocesado: (199364, 30)
📊 Shape de X_test preprocesado: (85443, 30)


In [ ]:
# 2. Configurar Sequential Forward Selection
sfs_selector = SequentialFeatureSelector(
    estimator=RandomForestClassifier(
        n_estimators=15,
        class_weight='balanced',
        random_state=42
    ),
    n_features_to_select=10,  # Seleccionar el 50% de características
    direction='forward',
    scoring='recall',  # 🎯 Optimizar para detectar fraudes
    cv=3,              # Menos folds para mayor velocidad
    n_jobs=-1
)

print("🚀 Ejecutando Sequential Forward Selection...")
sfs_selector.fit(X_train_preprocesado, y_train)

# 3. Obtener características seleccionadas
caracteristicas_seleccionadas = sfs_selector.get_support()
indices_seleccionados = np.where(caracteristicas_seleccionadas)[0]

print("✅ SFS COMPLETADO!")

🚀 Ejecutando Sequential Forward Selection...
✅ SFS COMPLETADO!


In [ ]:
print(caracteristicas_seleccionadas)

[False  True False False False False False False  True False False  True
 False  True  True  True  True False False False False  True  True False
 False False False False False  True]


In [ ]:
print(indices_seleccionados)

[ 1  8 11 13 14 15 16 21 22 29]


In [3]:
# Mapear índices a nombres de características
caracteristicas_seleccionadas_nombres = [nombres_caracteristicas[i] for i in indices_seleccionados]

print("🎯 VECTOR FINAL DE CARACTERÍSTICAS SELECCIONADAS")
print("=" * 50)
print(f"Total de características: {len(nombres_caracteristicas)}")
print(f"Características seleccionadas: {len(caracteristicas_seleccionadas_nombres)}")
print(f"Características descartadas: {len(nombres_caracteristicas) - len(caracteristicas_seleccionadas_nombres)}")


NameError: name 'indices_seleccionados' is not defined

In [7]:
print(f"\n✅ CARACTERÍSTICAS SELECCIONADAS POR SFS:")
for i, (idx, caracteristica) in enumerate(zip(indices_seleccionados, caracteristicas_seleccionadas_nombres), 1):
    print(f"   {i:2d}. {caracteristica} (Índice: {idx})")

print(f"\n📊 Características seleccionadas por tipo:")
pca_seleccionadas = [c for c in caracteristicas_seleccionadas_nombres if c.startswith('V')]
otras_seleccionadas = [c for c in caracteristicas_seleccionadas_nombres if not c.startswith('V')]

print(f"   - Componentes PCA: {len(pca_seleccionadas)}")
print(f"   - Otras variables: {len(otras_seleccionadas)}")
if otras_seleccionadas:
    print(f"     {otras_seleccionadas}")



✅ CARACTERÍSTICAS SELECCIONADAS POR SFS:


NameError: name 'indices_seleccionados' is not defined

## Entrenamiento de clasificadores

## Búsqueda y selección de hiperparámetros

## Evaluar el desempeño de los modelos